# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.12.1+cu113
cuda is available.


-----

## List up the models to check accuracy

In [4]:
model_names = [*set([
    'zbnwnfxx', 
    '1iaim0xt', 
    '2qlwjoct',
    '3n191i1l',
    '18eyvznu',
    '1smspudh',
    'h0byi1bz',
    'dejxbfxu',
    '2qd10pux',
    '20coz50b',
    '2o908z53',
    '1wk3lib1',
    '1gb8f2rq',
                     ])]

model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

['20coz50b',
 'h0byi1bz',
 '2qlwjoct',
 '2o908z53',
 '1gb8f2rq',
 '18eyvznu',
 'dejxbfxu',
 '2qd10pux',
 '3n191i1l',
 '1wk3lib1',
 '1iaim0xt',
 '1smspudh',
 'zbnwnfxx']


In [5]:
# model_pool = []

# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task2-ablation')

# for run in runs:
#     path = os.path.join(r'E:\CAUEEG\checkpoint', run.name, 'checkpoint.pt')
#     try:
#         ckpt = torch.load(path, map_location=device)
#         model_pool.append({'name': run.name, 'path': path})
#     except Exception as e:
#         print(e)
#         print(f'- {run.name}\'s checkpoint cannot be opened: {path}')
        
# pprint.pprint([model_dict['name'] for model_dict in model_pool])

---

## Configurations

In [6]:
base_repeat = 500
verbose = False

-----

## Load and check accuracy

In [7]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    
    if '220419' in config['dataset_path']:
        config['dataset_path'] = './local/dataset/caueeg-dataset/'
    config['run_mode'] = 'eval'
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    model_dict['activation'] = config.get('activation', 0)

    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure and update
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / config['crop_multiple'])
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = config['crop_multiple']
    model_dict['test_crop_multiple'] = config['test_crop_multiple']
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    
    # train accuracy
    # train_acc = check_accuracy(model, train_loader, 
    #                            config['preprocess_test'], config, repeat=repeat)
    # model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    val_acc = check_accuracy(model, val_loader, 
                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Validation Accuracy'] = val_acc
    
    # Test accuracy
    _ = check_accuracy_extended(model, test_loader, 
                                config['preprocess_test'], config, repeat=repeat)
    model_dict['Test Throughput'] = _[4]
    model_dict['Test Accuracy'] = _[0]
    test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'{k} ({c_name})'] = test_class_wise_metrics[k][c]
    
    # Multi-crop test accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                          config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Test Throughput'] = _[4]
    model_dict['Multi-Crop Test Accuracy'] = _[0]
    multi_test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]
            
print('==== Finished ====')

- checking for 20coz50b 1D-ResNet-18 ...
- checking for h0byi1bz 1D-Linear-SVM ...
- checking for 2qlwjoct 1D-Linear-SVM ...
- checking for 2o908z53 1D-ResNet-18 ...
- checking for 1gb8f2rq 1D-ResNet-18 ...
- checking for 18eyvznu 1D-Linear-SVM ...
- checking for dejxbfxu 1D-ResNet-18 ...
- checking for 2qd10pux 1D-ResNet-18 ...
- checking for 3n191i1l 1D-Linear-SVM ...
- checking for 1wk3lib1 1D-ResNet-18 ...
- checking for 1iaim0xt 1D-Linear-SVM ...
- checking for 1smspudh 1D-Linear-SVM ...
- checking for zbnwnfxx 1D-Linear-SVM ...
==== Finished ====


In [8]:
pprint.pprint(model_pool)

[{'Accuracy (Dementia)': 0.7701016949152543,
  'Accuracy (MCI)': 0.6529661016949152,
  'Accuracy (Normal)': 0.7879152542372881,
  'EKG': 'O',
  'F1-score (Dementia)': 0.5309171393000415,
  'F1-score (MCI)': 0.5036483964025115,
  'F1-score (Normal)': 0.7384023582045868,
  'Multi-Crop Accuracy (Dementia)': 0.7711864406779662,
  'Multi-Crop Accuracy (MCI)': 0.6468474576271186,
  'Multi-Crop Accuracy (Normal)': 0.7910508474576271,
  'Multi-Crop F1-score (Dementia)': 0.5336465386209756,
  'Multi-Crop F1-score (MCI)': 0.49254749147588894,
  'Multi-Crop F1-score (Normal)': 0.7431238539756626,
  'Multi-Crop Precision (Dementia)': 0.574360499702558,
  'Multi-Crop Precision (MCI)': 0.49182879377431904,
  'Multi-Crop Precision (Normal)': 0.7135083226632523,
  'Multi-Crop Sensitivity (Dementia)': 0.4983225806451613,
  'Multi-Crop Sensitivity (MCI)': 0.4932682926829268,
  'Multi-Crop Sensitivity (Normal)': 0.7753043478260869,
  'Multi-Crop Specificity (Dementia)': 0.8684137931034482,
  'Multi-Crop 

In [9]:
pd.DataFrame(model_pool)

,name,path,model,num_params,model size (MiB),seq_length,use_age,photic,EKG,awgn,...,Multi-Crop Sensitivity (Dementia),Multi-Crop Specificity (Normal),Multi-Crop Specificity (MCI),Multi-Crop Specificity (Dementia),Multi-Crop Precision (Normal),Multi-Crop Precision (MCI),Multi-Crop Precision (Dementia),Multi-Crop F1-score (Normal),Multi-Crop F1-score (MCI),Multi-Crop F1-score (Dementia)
0,20coz50b,E:\CAUEEG\checkpoint\20coz50b\checkpoint.pt,1D-ResNet-18,11394051,43.550922,2000,conv,O,O,0.004873,...,0.498323,0.801111,0.728623,0.868414,0.713508,0.491829,0.574360,0.743124,0.492547,0.533647
1,h0byi1bz,E:\CAUEEG\checkpoint\h0byi1bz\checkpoint.pt,1D-Linear-SVM,126006,0.481483,2000,fc,O,O,0.000000,...,0.308903,0.724111,0.700156,0.878529,0.654947,0.426926,0.475377,0.728101,0.423187,0.374472
2,2qlwjoct,E:\CAUEEG\checkpoint\2qlwjoct\checkpoint.pt,1D-Linear-SVM,126006,0.481483,2000,fc,O,O,0.000000,...,0.349935,0.720444,0.719584,0.862161,0.650264,0.425867,0.474956,0.722806,0.407490,0.402972
3,2o908z53,E:\CAUEEG\checkpoint\2o908z53\checkpoint.pt,1D-ResNet-18,11394051,43.550922,2000,conv,O,O,0.004873,...,0.538065,0.820667,0.771117,0.851218,0.749106,0.522540,0.563057,0.791102,0.495123,0.550277
4,1gb8f2rq,E:\CAUEEG\checkpoint\1gb8f2rq\checkpoint.pt,1D-ResNet-18,11394051,43.550922,2000,conv,O,O,0.004873,...,0.599226,0.819556,0.751169,0.857563,0.737514,0.489557,0.599845,0.764514,0.467964,0.599535
5,18eyvznu,E:\CAUEEG\checkpoint\18eyvznu\checkpoint.pt,1D-Linear-SVM,126006,0.481483,2000,fc,O,O,0.000000,...,0.124645,0.745667,0.574338,0.955034,0.656409,0.424255,0.496914,0.704641,0.493260,0.199299
6,dejxbfxu,E:\CAUEEG\checkpoint\dejxbfxu\checkpoint.pt,1D-ResNet-18,11394051,43.550922,2000,conv,O,O,0.004873,...,0.553032,0.831222,0.774026,0.867586,0.754207,0.563866,0.598102,0.781391,0.556171,0.574685
7,2qd10pux,E:\CAUEEG\checkpoint\2qd10pux\checkpoint.pt,1D-ResNet-18,11394051,43.550922,2000,conv,O,O,0.004873,...,0.485935,0.808667,0.722182,0.880460,0.708629,0.527311,0.591580,0.718353,0.553330,0.533579
8,3n191i1l,E:\CAUEEG\checkpoint\3n191i1l\checkpoint.pt,1D-Linear-SVM,126006,0.481483,2000,fc,O,O,0.000000,...,0.341677,0.713444,0.720831,0.866943,0.645010,0.429996,0.477806,0.720092,0.412034,0.398435
9,1wk3lib1,E:\CAUEEG\checkpoint\1wk3lib1\checkpoint.pt,1D-ResNet-18,11394051,43.550922,2000,conv,O,O,0.004873,...,0.442065,0.767333,0.727896,0.879816,0.685160,0.484347,0.567219,0.734941,0.482164,0.496882


In [10]:
pd.DataFrame(model_pool).to_csv('local/output/caueeg-dementia-svm__.csv')

PermissionError: [Errno 13] Permission denied: 'local/output/caueeg-dementia-svm__.csv'